In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime 
import os.path
from os import path
import seaborn as sns
from scipy import signal

In [488]:
## 1, -- KDP_nu
KDP_nu_name=["CIR40" ,"CIR50"]
KDP_nu=[1,1.25]

## 2,  --  KDP_w_sat_1  -- -
KDP_w_sat_1_name=["0_33" ,"0_66" ,"1_00"]
KDP_w_sat_1=[0.33, 0.66 ,1.00]

## 3,  --  KDP_w_sat_2  -- -
KDP_w_sat_2_name=["ImmunEscp_Jul" ,"ImmunEscp_Sep", "ImmunEscp_Nov"]
KDP_w_sat_2=[380, 442 ,503]

## 4,  --  KDP_g_1  -- -
KDP_g_1_name=["0_33", "0_66" ,"1_00"]
KDP_g_1=[0.33, 0.66, 1.00]

## 5,  --  KDP_g_2  -- -
KDP_g_2_name=["ABW150Days" ,"ABW180Days"]
KDP_g_2=[150 ,180]

## 6,  --  KDP_c_p  -- -
KDP_c_p_name=["Vaccine_PresentRate", "Vaccine_BestWeekAvg", "Vaccine_TwicePresent"]
KDP_c_p=[1 ,1.5, 2]

KDP_lambda_name=["SD_RA","SD_RAR","SD_RARF"]
KDP_lambda=[0,0.1,0.2]

### CODE FOR OBTAINING DATE VALUES ####################
start ="2020-07-01"
end  = "2022-06-30"

start = datetime.datetime.strptime(start, "%Y-%m-%d")
end = datetime.datetime.strptime(end, "%Y-%m-%d")
date_generated = [start + datetime.timedelta(days=x) for x in range(0, (end-start).days)]



for i in range(len(date_generated)):
    date_generated[i] = date_generated[i].strftime("%d-%b-%y")


In [80]:
def ReadDF(fname):
    df_master = pd.DataFrame()
    dateOfPeak = []
    dayOfPeak = []
    numAtPeak = []
    vaccRate = []
    numRead = 0
    for i in range(len(KDP_nu)):
            for j in range(len(KDP_w_sat_1)):
                for k in range(len(KDP_w_sat_2)):
                    for l in range(len(KDP_g_1_name)):
                        for m in range(len(KDP_g_2)):
                            for n in range(len(KDP_c_p)):
                                for o in range(len(KDP_lambda)):
                                    if(path.exists(KDP_nu_name[i] + "_" + KDP_w_sat_1_name[j] + "_" + KDP_w_sat_2_name[k] + "_" + \
                                                KDP_g_1_name[l] + "_" + KDP_g_2_name[m] + "_" + KDP_c_p_name[n] + "_" + \
                                                KDP_lambda_name[o]+"/PopulationData/"+ fname)):
                                        df_temp = pd.read_csv(KDP_nu_name[i] + "_" + KDP_w_sat_1_name[j] + "_" + KDP_w_sat_2_name[k] + "_" + \
                                                KDP_g_1_name[l] + "_" + KDP_g_2_name[m] + "_" + KDP_c_p_name[n] + "_" + \
                                                KDP_lambda_name[o]+"/PopulationData/"+fname, delim_whitespace=True, header=1).transpose().loc['Total':]

                                        

                                        ExptId = KDP_nu_name[i] + "_" + KDP_w_sat_1_name[j] + "_" + KDP_w_sat_2_name[k] + "_" + \
                                                KDP_g_1_name[l] + "_" + KDP_g_2_name[m] + "_" + KDP_c_p_name[n] + "_" + \
                                                KDP_lambda_name[o]
                                        VaccineRt = str(KDP_c_p_name[n])
                                        CIR = str(KDP_nu_name[i]) 
                                        SD = str(KDP_lambda_name[o])
                                        KDPwsat = str(KDP_w_sat_2_name[k])

                                        numRead+=1


                                        df_temp['ID'] = ExptId
                                        df_temp['Vaccine Rate'] = VaccineRt
                                        df_temp['CIR'] = CIR
                                        df_temp['SD'] = SD
                                        df_temp['KDPwsat2']=KDPwsat
                                        df_temp['Scenario'] = numRead
                                        
                                                                
                                        df_master = pd.concat([df_master,df_temp])


    print(str(numRead) + " samples have been read ")
    return df_master
                                    

In [494]:



#### - Read the Nucleation Data
df_nucleation_total = ReadDF("CovidNucleation.data")
df_nucleation_11 = ReadDF("CovidNucleation_11Yrs.data")
df_nucleation_18 = ReadDF("CovidNucleation_18Yrs.data")
df_nucleation_45 = ReadDF("CovidNucleation_45Yrs.data")
df_nucleation_60 = ReadDF("CovidNucleation_60Yrs.data")
df_nucleation_60plus = ReadDF("CovidNuclthegreaeation_Above60Yrs.data")


## Read the VaccinationData
#### Read the Vaccination and the Antibody plots

df_antibody = ReadDF("Covid_Vaccinated.data")
df_Vaccinated = ReadDF("Covid_Antibody.data")


972 samples have been read 
972 samples have been read 
972 samples have been read 
972 samples have been read 
972 samples have been read 
972 samples have been read 
972 samples have been read 
972 samples have been read 
972 samples have been read 


In [519]:
################# --- READ THE MASTER DATA ------------- ####################################

df_active= ReadDF("CovidPopulation.data")

972 samples have been read 


In [496]:
################ THIVIN ####################################
### Compute New dataframes for the ratio Quantities    -- Nucleation_age/Nucleation_Total

############# HARDCODED ##########################
##  Total of 730 days simulated 
ROWS = 730

### getList of columns
listOfCols = list(df_nucleation_total.columns.values)

#### create a copy of all the dataframes to store the ratio values
df_nucleation_11_ratio = df_nucleation_11.copy(deep=True)
df_nucleation_18_ratio = df_nucleation_18.copy(deep=True)
df_nucleation_45_ratio = df_nucleation_45.copy(deep=True)
df_nucleation_60_ratio = df_nucleation_60.copy(deep=True)
df_nucleation_60plus_ratio = df_nucleation_60plus.copy(deep=True)


##### Compute Ratios ##################

for i in range(ROWS + 1):
    df_nucleation_11_ratio[[i]] =  df_nucleation_11_ratio[[i]].div(df_nucleation_total[[i]],axis=1) 
    df_nucleation_18_ratio[[i]] =  df_nucleation_18_ratio[[i]].div(df_nucleation_total[[i]],axis=1) 
    df_nucleation_45_ratio[[i]] =  df_nucleation_45_ratio[[i]].div(df_nucleation_total[[i]],axis=1) 
    df_nucleation_60_ratio[[i]] =  df_nucleation_60_ratio[[i]].div(df_nucleation_total[[i]],axis=1) 
    df_nucleation_60plus_ratio[[i]] =  df_nucleation_60plus_ratio[[i]].div(df_nucleation_total[[i]],axis=1) 

    
## remove the NA values
df_nucleation_11_ratio = df_nucleation_11_ratio.replace(np.nan,0)
df_nucleation_18_ratio = df_nucleation_18_ratio.replace(np.nan,0)
df_nucleation_45_ratio = df_nucleation_45_ratio.replace(np.nan,0)
df_nucleation_60_ratio = df_nucleation_60_ratio.replace(np.nan,0)
df_nucleation_60plus_ratio = df_nucleation_60plus_ratio.replace(np.nan,0)



In [479]:

def plot_histo(df,X,hue,bw):
    df_plot=df
    df_plot.dropna(subset=['Date Of Peak'],inplace=True)
    sns.set(rc={'figure.figsize':(20,15)})
    sns.histplot(data=df_plot,x=X, hue=hue, binwidth=bw,multiple='dodge',fill=True)
    
    
def plot_violin(X,Y,df,hue,scale,cut,bandwidth):
    df_plot = df
    df_plot.dropna(subset=['Date Of Peak'],inplace=True)
    df_plot = df_plot.loc[df_plot[Y]< 4000000]
    sns.set(rc={'figure.figsize':(15,10)})
    sns.violinplot(x=X,y=Y,data=df_plot,hue=hue,split=True,scale=scale,cut=cut,bw=bandwidth)

def plot_catplot(X,Y,df,hue,scale,cut,bandwidth,xticks):
    df_plot = df
    df_plot.dropna(subset=['Date Of Peak'],inplace=True)
    df_plot = df_plot.loc[df_plot[Y]< 4000000]
#     sns.set(rc={'figure.figsize':(15,10)})
    fig = sns.catplot(x=X, y=Y,
                hue=hue, col="Vaccine Rate",data=df_plot,
                kind="violin", split=True,
                height=4, aspect=.9,scale=scale,cut=cut,bw=bandwidth);
    fig.set_xticklabels(xticks)

def timeSeriesPlot(X,Y,df,hue,scale,cut):
    df_plot = df
    df_plot.dropna(subset=['Date Of Peak'],inplace=True)
    df_plot = df_plot.loc[df_plot[Y]< 10]
    print(df_plot)
#     sns.set(rc={'figure.figsize':(15,10)})
    sns.catplot(x=X, y=Y,
                hue=hue, col="Vaccine Rate",data=df_plot,
                kind="violin", split=True,
                height=4, aspect=.9,scale=scale,cut=cut,bw=bandwidth);

In [524]:
##########--- Compute MeanData ----- ######
def AddMeanInfo(df):
    meanval = []
    minval = []
    maxval = []
    for i in range(731):
        meanval.append(np.mean(df[i]))
        minval.append(np.min(df[i]))
        maxval.append(np.max(df[i]))
    df.append(minval)
    df.append(maxval)
    df.append(meanval)
    

import scipy
def AddPeakInfo(df_master):
    dateOfPeak = []
    numAtPeak = []
    dateString = [] 
    for ia in range(len(df_master)):
        dop = scipy.signal.find_peaks(df_master.iloc[[ia]].transpose().loc[0:730,'Total'],distance = 20)
        for ib in range(len(dop[0])):
            if (dop[0][ib]>365):
                dateOfPeak.append(dop[0][ib])
                numAtPeak.append(df_master.iloc[ia,dop[0][ib]])
                dateString.append(date_generated[dop[0][ib]])
                break
            if(ib)==len(dop[0])-1:
                #print("Scenario " + str(ia) + " Does Not Have a Peak ")
                dateOfPeak.append(float('nan'))
                numAtPeak.append(float('nan'))
                dateString.append(float('nan'))
    df_master['Date Of Peak'] = dateOfPeak
    df_master['Number At Peak'] = numAtPeak
    df_master['DateString'] = dateString
    return df_master
        
        
    

In [525]:
### -- Compute the Peak Data of the Time Series --- #####

df_active = AddPeakInfo(df_active)

### -- Compute the Mean Data of the Time Series --- #####
# df_active = AddMeanInfo(df_active)

In [562]:
### PLOT THE OUTLIERS FOR BI MODAL PLOTS ######

KDPSAT2      = "ImmunEscp_Nov"
VACCINERATE  = "Vaccine_PresentRate"
CIRR         = "CIR40"

OutlierNov = df_active.loc[(df_active['KDPwsat2'] == KDPSAT2) &  (df_active['Vaccine Rate'] == VACCINERATE) & (df_active['CIR'] == CIRR)].copy(deep=True)

result  = OutlierNov.dropna(subset=['Date Of Peak'], inplace=False)
pd.set_option('display.max_rows', None, 'display.max_columns', None)
pd.set_option('display.max_colwidth', None)

print(result.loc[result['Date Of Peak'] >= 545][['ID','Date Of Peak','Number At Peak','DateString']])

                                                                        ID  Date Of Peak  Number At Peak DateString
Total  CIR40_0_33_ImmunEscp_Nov_0_33_ABW150Days_Vaccine_PresentRate_SD_RAR         604.0        188217.0  25-Feb-22
Total  CIR40_0_33_ImmunEscp_Nov_0_33_ABW180Days_Vaccine_PresentRate_SD_RAR         604.0        188217.0  25-Feb-22
Total  CIR40_0_33_ImmunEscp_Nov_0_66_ABW150Days_Vaccine_PresentRate_SD_RAR         603.0        192173.0  24-Feb-22
Total  CIR40_0_33_ImmunEscp_Nov_0_66_ABW180Days_Vaccine_PresentRate_SD_RAR         603.0        192173.0  24-Feb-22
Total  CIR40_0_33_ImmunEscp_Nov_1_00_ABW150Days_Vaccine_PresentRate_SD_RAR         602.0        196484.0  23-Feb-22
Total  CIR40_0_33_ImmunEscp_Nov_1_00_ABW180Days_Vaccine_PresentRate_SD_RAR         602.0        196484.0  23-Feb-22
Total  CIR40_0_66_ImmunEscp_Nov_0_33_ABW150Days_Vaccine_PresentRate_SD_RAR         601.0        193934.0  22-Feb-22
Total  CIR40_0_66_ImmunEscp_Nov_0_33_ABW180Days_Vaccine_PresentRate_SD_R